## Columbia University
### ECBM E4040 Neural Networks and Deep Learning. Fall 2021.

# ECBM E4040 - Assignment 2- Task 5: Kaggle Open-ended Competition

Kaggle is a platform for predictive modelling and analytics competitions in which companies and researchers post data and statisticians and data miners compete to produce the best models for predicting and describing the data.

If you don't have a Kaggle account, feel free to join at [www.kaggle.com](https://www.kaggle.com). To let the CAs do the grading more conveniently, please __use Lionmail to join Kaggle__ and __use UNI as your username__.

The competition is located here: https://www.kaggle.com/c/ecbm4040-assignment-2-task-5/overview.

You can find detailed description about this in-class competition on the website above. Please read carefully and follow the instructions.

<span style="color:red">__TODO__:</span>
1. Train a custom model for the bottle dataset classification problem. You are free to use any methods taught in the class or found by yourself on the Internet (ALWAYS provide reference to the source).
General training methods include:
    * Dropout
    * Batch normalization
    * Early stopping
    * l1-norm & l2-norm penalization
    
2. You are given the test set to generate your predictions (70% public + 30% private, but you don't know which ones are public/private). Students should achieve an accuracy on the public test set of at least 70%. Two points will be deducted for each 1% below 70% accuracy threshold (i.e. 65% accuracy will have 10 points deducted). The accuracy will be shown on the public leaderboard once you submit your prediction .csv file. The private leaderboard will be released after the competition. The final ranking is based on the private leaderboard result, not the public leaderboard.
3. 

    * Report your results on the Kaggle, for comparison with other students' optimal results (you can do this several times). 
    * Save your best model, using Github Classroom, at the same time when you submit the homework files into Courseworks. See instructions below. 

__Hint__: You can start from what you implemented in task 4. Another classic classification model named 'VGG16' can also be easily implemented. Students are allowed to use pretrained networks, and utilize transfer learning. 

## HW Submission Details:
There are three components to reporting the results of this task: 

**(A) Submission (up to 20 submissions each day) of the .csv prediction file throught the Kaggle platform;**. You should start doing this __VERY early__, so that students can compare their work as they are making progress with model optimization.

**(B) Editing and submitting the content of this Jupyter notebook, through Courseworks; **
(i) The code for your CNN model and for the training function. The code should be stored in __./ecbm4040/neuralnets/kaggle.py__;
(ii) Print out your training process and accuracy __within this notebook__;

**(C) Submitting your best CNN model through Github Classroom repo.**

**Description of (C):** 
For this task, we will continue to use Github classroom to save your model for submission. 

<span style="color:red">__Submission content:__ :</span>
(i) In your Assignment 2 submission folder, create a subfolder called __KaggleModel__. Upload your best model with all the data output (for example, __MODEL.data-00000-of-00001, MODEL.meta, MODEL.index__) into the folder. 
(ii) Remember to delete any intermediate results, **we only want your best model. Do not upload any data files**. The instructors will rerun the uploaded best model and verify against the score which you reported on the Kaggle.



## Load Data

In [3]:
import zipfile
import os
files = zipfile.ZipFile('ecbm4040-assignment-2-task-5.zip', 'r')
files.extractall(os.getcwd())
files.close()

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator,array_to_img,img_to_array,load_img

# create generator and data augmentation
datagen = ImageDataGenerator(validation_split=0.2, 
                             horizontal_flip=True,
                             width_shift_range=[-20,20],
                             rescale=1./255,
                             rotation_range=40)

# datagen = ImageDataGenerator(validation_split=0.2)

# prepare an iterators for each dataset
directory_train = 'kaggle_train_128/train_128/'
directory_test = 'kaggle_test_128/'

# train set
train_bottle = datagen.flow_from_directory(
    directory=directory_train, 
    target_size=(128, 128),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42,
    subset='training')

# validation set
validation_bottle = datagen.flow_from_directory(
    directory=directory_train,  # same directory as training data
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation') # set as validation data

# test set
test_bottle = datagen.flow_from_directory(
    directory=directory_test,
    target_size=(128, 128),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    )

# confirm the iterator works
batchX, batchy = train_bottle.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Found 12000 images belonging to 5 classes.
Found 3000 images belonging to 5 classes.
Found 3500 images belonging to 1 classes.
Batch shape=(32, 128, 128, 3), min=0.000, max=1.000


## Build and Train Your Model Here

In [1]:
# YOUR CODE HERE
import pandas as pd
import os
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from utils.cifar_utils import load_data
import numpy as np

## Save your best model

In [6]:
# YOUR CODE HERE
STEP_SIZE_TRAIN = train_bottle.n // train_bottle.batch_size
STEP_SIZE_VALID = validation_bottle.n // validation_bottle.batch_size
print(STEP_SIZE_TRAIN, STEP_SIZE_VALID)

375 93


## Generate .csv file for Kaggle

The following code snippet can be used to generate your prediction .csv file.

NOTE: If your kaggle results are indicating random performance, then it's likely that the indices of your csv predictions are misaligned.

In [ ]:
# import csv
# with open('predicted.csv','w') as csvfile:
#     fieldnames = ['Id','label']
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#     writer.writeheader()
#     for index,l in enumerate(predicted_values_generated_by_your_model):
#         filename = str(index) + '.png'
#         label = str(l)
#         writer.writerow({'Id': filename, 'label': label})

In [3]:
from tensorflow.keras.applications import VGG19

In [4]:
base_model=VGG19(weights='imagenet',include_top=False) 
#I choose VGG19 as transfer learning model
from tensorflow.keras.layers import Dropout, BatchNormalization
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu', name='Dense1')(x)
x=Dropout(0.7)(x)
x=Dense(1024,activation='relu', name='Dense2')(x) 
x=Dropout(0.7)(x)
x=Dense(512,activation='relu', name='Dense3')(x)
preds=Dense(5,activation='softmax')(x)

In [5]:

model=Model(inputs=base_model.input,outputs=preds,name='Kaggle_model')

In [6]:
from tensorflow.keras.models import Sequential
updated_model = Sequential()
i = 1
for layer in model.layers:
    updated_model.add(layer)
    if layer.name in ['conv2', 'conv2', 'conv4', 'conv4', 'conv4']:
        updated_model.add(BatchNormalization(name='bn_{}'.format(i)))
        i += 1

model = updated_model

In [7]:
for layer in model.layers[:11]:
    layer.trainable=False
for layer in model.layers[11:]:
    layer.trainable=True

In [8]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, None, None, 256)   2

In [13]:
epc = 25 #number of epoches

# train the model
history = model.fit_generator(generator=train_bottle,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_bottle,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=25
)

/home/ecbm4040/envTF24/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
375/375 [==============================] - 104s 202ms/step - loss: 1.5750 - accuracy: 0.2989 - val_loss: 2.3585 - val_accuracy: 0.5901
Epoch 2/25
375/375 [==============================] - 75s 200ms/step - loss: 0.8229 - accuracy: 0.7204 - val_loss: 0.6510 - val_accuracy: 0.7907
Epoch 3/25
375/375 [==============================] - 75s 200ms/step - loss: 0.5618 - accuracy: 0.8162 - val_loss: 0.7305 - val_accuracy: 0.7927
Epoch 4/25
375/375 [==============================] - 75s 201ms/step - loss: 0.4669 - accuracy: 0.8512 - val_loss: 0.5028 - val_accuracy: 0.8488
Epoch 5/25
375/375 [==============================] - 75s 201ms/step - loss: 0.4080 - accuracy: 0.8696 - val_loss: 0.4150 - val_accuracy: 0.8629
Epoch 6/25
375/375 [==============================] - 75s 200ms/step - loss: 0.3742 - accuracy: 0.8800 - val_loss: 0.5764 - val_accuracy: 0.8253
Epoch 7/25
375/375 [==============================] - 75s 201ms/step - loss: 0.3226 - accuracy: 0.9005 - val_loss: 0.3820 - val_a

In [14]:
model.evaluate_generator(generator=validation_bottle,
steps=STEP_SIZE_VALID)

/home/ecbm4040/envTF24/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


[0.3125428259372711, 0.9200268983840942]

In [15]:
STEP_SIZE_TEST=test_bottle.n//test_bottle.batch_size
test_bottle.reset()
pred=model.predict_generator(test_bottle,
steps=STEP_SIZE_TEST,
verbose=1)

/home/ecbm4040/envTF24/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


3500/3500 [==============================] - 25s 7ms/step


In [16]:
predicted_class_indices=np.argmax(pred,axis=1)

In [17]:

labels = (train_bottle.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [18]:
model.save('KaggleModel/KaggleModel.yc3993')

INFO:tensorflow:Assets written to: KaggleModel/KaggleModel.yc3993/assets


In [41]:
filenames=[x for x in test_bottle.filenames]
results=pd.DataFrame({"Id":filenames,
                      "label":predictions})
results.to_csv("prediction_label.csv",index=False)